## What is Pearson correlation?

Given any two random variables $X$ and $Y$ associated to a population, the **Pearson correlation** is:


$\rho_{XY} = \Large\frac{cov(X,Y)}{\sigma_{X}\sigma_{Y}} = \frac{\mathbb{E}[(X - \mu_{X})(Y - \mu_{Y})]}{\sigma_{X}\sigma_{Y}}$


When dealing with a sample (i.e. when we have a finite set of records which capture only a portion of the whole population of intereset, like in almost every dataset), the Pearson correlation is instead identified with:

$r_{XY} = \Large\frac{\sum_{i}(x_{i} - \bar{x})(y_{i} - \bar{y})}{\sqrt{\sum_{i}(x_{i} - \bar{x})^2} \sqrt{\sum_{i} (y_{i} - \bar{y})^2}}$

### Why do we need approximate methods to search across a large collection of tables?

Computing correlation is a **costly operation**: it involves to align two variables (i.e. _join_ two datasets, if the variables are taken from different sources) and compute the _standard deviation_ and their _correlation_.

Doing these steps with a high number of datasets may be unfeasible with limited resources.

The **Quadrant Count Ratio** schema approximates the Pearson correlation, and it's faster and easier to compute than it on large scale.

However, it's also generally less accurate.

## Use BLEND to search for correlated variables

### Load libraries and define paths

In [ ]:
import os
import sys
from pathlib import Path
import polars as pl
from tabulate import tabulate

In [ ]:
data_path = Path("..", "data", "undata")

data_path.absolute(), data_path.exists()

In [ ]:
modules_path = Path("..", "modules")
blend_module_path = modules_path.joinpath("BLEND")

sys.path.append(str(blend_module_path.resolve()))
sys.path

In [ ]:
db_path = data_path.joinpath("index_blend.db")
data_lake_path = data_path.joinpath("data-lake")
queries_path = data_path.joinpath("queries")

In [ ]:
db_path.exists()

In [ ]:
from blend import BLEND
from blend.utils import clean

### Load the BLEND index

In [ ]:
index = BLEND(db_path)

### Load the query dataset

We have some datasets in the _query_ folder:

In [ ]:
queries = sorted(os.listdir(queries_path))

print('\n\n'.join(queries))

## Join-Correlation Search 

Given a query dataset composed by two columns, _Kq_ and _Xq_, 
we need to identify datasets on which we can perform a join on key _Kc_ and that have a numerical column _Xc_ that is highly correlated with _Xq_.

Just looking for columns with an high join-overlap doesn't address our needs.

Define which dataset we want to use:

In [ ]:
query_table_idx = 2
query_table_name = queries[query_table_idx]

qdf = pl.read_csv(os.path.join(queries_path, query_table_name))

print(f"Query dataset: {query_table_name}")

qdf

Define on which key and target columns we will perform the search

In [ ]:
target_column_name = 'Value'

target_column_name

In [ ]:
# key_column_name = 'Country or Area'
key_column_name = 'Sub-region Name'
# key_column_name = 'Region Name'

key_column_name

### What's inside the key column?

When working with joins, correlations, ..., the granularity level choosen for the search affects the final results.

In our geographical datasets, fine-grained searches at the country level yield different results compared to coarser-grained searches at sub-regional or regional level.

In particular, regional granularity isn't really useful.

In [ ]:
qdf.get_column(key_column_name).unique().sort()

### Perform a Correlation Search (based on QCR schema)

Run a correlation search on the query dataset. 

First, we group it by the selected key column, which will be used to identify _joinable_ tables.

Then the retrieved tables will be ranked by an _estimate_ of the Pearson correlation, called **Quadrant Count Ratio** (QCR) correlation

In [ ]:
# GROUP BY key_column + MEAN ON target_column
grouped_qdf = qdf.group_by(key_column_name).agg(pl.col(target_column_name).mean())

# rename, just because it will be useful in later steps
grouped_qdf = grouped_qdf.rename({target_column_name: 'Value_left'})

grouped_qdf.sort(key_column_name)

In [ ]:
# extract the key column values
keys = grouped_qdf.get_column(key_column_name).map_elements(clean, pl.String).to_list()

# extract the target column values
targets = grouped_qdf.get_column('Value_left').to_list()

keys[:3], targets[:3]

Run the correlation search task: see the relative SQL query used under the hood at **blend.Operators.Seekers.Correlation**

In [ ]:
results = index.correlation_search(keys, targets, 20)

results_df = pl.DataFrame(results, schema=['dataset', 'join_col_idx', 'target_col_idx', 'QCR'], orient='row')

results_df

### Compare results with actual Pearson

In [ ]:
from scipy import stats


def compare_with_pearson(results: list) -> list:
    results_with_pearson = []

    # basically, for each record load the relative dataset and compute 
    # the exact Pearson correlation (we have all necessary information, 
    # from the column indexes for key and target columns to the actual dataset)
    for table_id, join_col_idx, target_col_idx, qcr in results:
        r_df = pl.scan_csv(data_lake_path.joinpath(f"{table_id}.csv"))

        # aggregate each dataset on its identified key column 
        # and compute the mean of the group
        r_df = r_df.group_by(pl.nth(join_col_idx)).agg(pl.nth(target_col_idx).mean()).collect()
        
        # rename the column (just to make simpler next steps)
        target_col_name = r_df.columns[1]
        r_df = r_df.rename({r_df.columns[0]: key_column_name, r_df.columns[1]: 'Value_right'})
        
        # join our query grouped dataframe with the retrieved one
        # (which is, as well, grouped)
        join = grouped_qdf.join(
            r_df, 
            on=key_column_name
        )

        # extract the numerical columns used to compute the correlation
        value_left = join.get_column('Value_left')
        value_right = join.get_column('Value_right')

        # it could happen that after aggregation an array is
        # constant: in this case, Pearson correlation is not
        # defined and in the end we will have to discard these
        # NaN values...
        statistics = stats.pearsonr(value_left, value_right)

        pearson = statistics.correlation
        p_value = statistics.pvalue

        results_with_pearson.append(
            [
                table_id, join_col_idx, target_col_idx, target_col_name, qcr, pearson, p_value
            ]
        )

    return results_with_pearson

In [ ]:
results_with_pearson = compare_with_pearson(results)

results_with_pearson_df = pl.DataFrame(
    results_with_pearson, 
    schema=['dataset', 'join_col_idx', 'target_col_idx', 'target_col_name', 'QCR', 'pearson', 'p_value'], 
    orient='row'
    ).with_row_index('rank')

results_with_pearson_df

In [ ]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, HoverTool, Slider, CustomJS
from bokeh.transform import factor_cmap
from bokeh.layouts import row

# Display Bokeh plots in Jupyter
output_notebook()

In [ ]:
# Categorize p-values
def categorize_pval(p):
    if p < 0.05:
        return "< 0.05"
    elif p < 0.5:
        return "0.05-0.5"
    else:
        return ">=0.5"

df_pd = results_with_pearson_df.with_columns(
    pl.col('p_value').map_elements(categorize_pval, pl.String).alias('p_category')
).to_pandas()

# Convert to Bokeh ColumnDataSource
source = ColumnDataSource(df_pd)
source_all = ColumnDataSource(df_pd)

# Define color mapping for categories
categories = ["< 0.05", "0.05-0.5", ">=0.5"]
colors = ["red", "orange", "blue"]

# Create interactive plot
p = figure(
    title="Estimated vs Actual Pearson (interactive)",
    x_axis_label="Pearson",
    y_axis_label="Estimated Pearson",
    width=700,
    height=700,
    tools="pan,wheel_zoom,box_zoom,reset"
)

# Add scatter with hover tool
renderer = p.scatter(
    x="pearson",
    y="QCR",
    source=source,
    size=8,
    legend_field="p_category",
    fill_alpha=0.7,
    color=factor_cmap("p_category", palette=colors, factors=categories)
)

# Add identity line y=x
p.line([-1, 1], [-1, 1], line_dash="dashed", line_color="black")

# Add hover tool
hover = HoverTool(
    renderers=[renderer],
    tooltips=[
        ("Pearson", "@pearson{0.000}"),
        ("Estimate", "@QCR{0.000}"),
        ("p-value", "@p_value"),
        ("p-category", "@p_category"),
        ("target_name", "@target_col_name"),
        ("dataset", "@dataset"),
        ("rank", "@rank")
    ]
)
p.add_tools(hover)

p.legend.title = "p-value bins"
p.legend.location = "top_left"


# Slider widget for filtering by rank
slider = Slider(start=df_pd["rank"].min(),
                end=df_pd["rank"].max(),
                value=df_pd["rank"].max(),
                step=1,
                title="Max rank in top-K")

# Reassign the ENTIRE data dict
callback = CustomJS(args=dict(source=source, source_all=source_all, slider=slider), code="""
    const A = source_all.data;
    const max_pos = slider.value;

    const idx = [];
    const n = A['rank'].length;
    for (let i = 0; i < n; i++) {
        if (A['rank'][i] < max_pos) idx.push(i);
    }

    function pick(key) { return idx.map(i => A[key][i]); }

    source.data = {
        pearson: pick('pearson'),
        QCR: pick('QCR'),
        p_value: pick('p_value'),
        p_category: pick('p_category'),
        target_col_name: pick('target_col_name'),
        dataset: pick('dataset'),
        rank: pick('rank'),
    };
""")

slider.js_on_change("value", callback)

# Layout (plot + slider)
layout = row(p, slider)

print(f"Query dataset: {query_table_name}")
show(layout)

The QCR scheme gives an approximation of the Pearson correlation between two numerical variables. However, before going on with any analysis, the p-value should be checked to verify that the correlation could be considered statistically significant, and not spurious.

In general, this is a **pure statistical method**, and no information on the actual meaning of a variable is involved. Thus, a final check is always required before using the retrieved columns.

# Exercise: Compute Correlation on More than one key column

What if we need as key a combination of two or more elements? The QCR schema cannot address this scenario in its basic formula

In [ ]:
query_table_idx = 0
query_table_name = queries[query_table_idx]

qdf = pl.read_csv(os.path.join(queries_path, query_table_name))

print(f"Query dataset: {query_table_name}")

qdf

We can run a join-correlation query on every key attribute and the aggregate the final results:

In [ ]:
# Run a join-correlation query against the "Sub-region Name" column
key_column_name = "Sub-region Name"
target_column_name = "Value"

grouped_qdf = qdf.group_by(key_column_name).agg(pl.col(target_column_name).mean())
grouped_qdf = grouped_qdf.rename({target_column_name: 'Value_left'})

keys = grouped_qdf.get_column(key_column_name).map_elements(clean, pl.String).to_list()
targets = grouped_qdf.get_column('Value_left').to_list()

results_subregion = index.correlation_search(keys, targets, 20)
results_df__subregion = pl.DataFrame(results_subregion, schema=['dataset', 'join_col_idx', 'target_col_idx', 'QCR'], orient='row')

In [ ]:
results_with_pearson_df_subregion = pl.DataFrame(
    compare_with_pearson(results_subregion), 
    schema=['dataset', 'join_col_idx', 'target_col_idx', 'target_col_name', 'QCR', 'pearson', 'p_value'], 
    orient='row'
    ).with_row_index('rank').filter(pl.col("target_col_name").is_in(["Year", "Decade"]).not_())

In [ ]:
# Run a join-correlation query against the "Decade" column
key_column_name = "Decade"
target_column_name = "Value"

grouped_qdf = qdf.group_by(key_column_name).agg(pl.col(target_column_name).mean())
grouped_qdf = grouped_qdf.rename({target_column_name: 'Value_left'})

keys = grouped_qdf.get_column(key_column_name).map_elements(clean, pl.String).to_list()
targets = grouped_qdf.get_column('Value_left').to_list()

results_decade = index.correlation_search(keys, targets, 20)
results_df__decade = pl.DataFrame(results_decade, schema=['dataset', 'join_col_idx', 'target_col_idx', 'QCR'], orient='row')

In [ ]:
results_with_pearson_df_subregion.head()

In [ ]:
results_with_pearson_df_decade = pl.DataFrame(
    compare_with_pearson(results_decade), 
    schema=['dataset', 'join_col_idx', 'target_col_idx', 'target_col_name', 'QCR', 'pearson', 'p_value'], 
    orient='row'
    ).with_row_index('rank').filter(pl.col("target_col_name").is_in(["Year", "Decade"]).not_())

In [ ]:
results_with_pearson_df_decade.head()

What are the datasets that appear in both result sets? What is the real Pearson correlation? 

If we instead join the datasets on the two columns and then compute the correlation, does this value
differ from the previously computed ones?

In [ ]:
datasets_subregion = results_with_pearson_df_subregion.get_column("dataset").to_list()
datasets_decade = results_with_pearson_df_decade.get_column("dataset").to_list()

In [ ]:
common_datasets = set(datasets_subregion).intersection(datasets_decade)
common_datasets

Otherwise, we can create a custom key on all the datasets for which we can recognize the desired key columns. 